# Car accidents in Berlin, Germany, in 2019 

Public data is described at [Daten.Berlin.de](https://daten.berlin.de/datensaetze/strassenverkehrsunf%C3%A4lle-nach-unfallort-berlin-2019)

In [1]:
INPUT_FOLDER_NAME = "/Users/abuzatu/Work/data/geo_spatial/Germany/Berlin/Accidents"
INPUT_FILE_NAME = f"{INPUT_FOLDER_NAME}/AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2019_Datensatz.csv"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import geopandas as gpd

In [3]:
df = pd.read_csv(f"{INPUT_FILE_NAME}", sep=";", encoding = "ISO-8859-1")
df

,OBJECTID,LAND,BEZ,LOR,STRASSE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,USTRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,49090,11,12,12301203,Wittenau Süd,2019,1,13,6,3,...,1,0,0,0,0,1,"794062,2837","5835083,823","13,34146","52,58609"
1,49091,11,3,3040818,Pankow Süd,2019,1,9,5,3,...,1,0,0,0,0,0,"799130,4007","5832327,415","13,41356","52,55862"
2,49093,11,12,12103115,Breitkopfbecken,2019,3,21,6,3,...,0,0,0,0,0,0,"795437,613","5833549,454","13,36034","52,57159"
3,49096,11,6,6040703,Nikolassee,2019,1,7,6,2,...,1,1,0,0,0,1,"786714,3754","5817042,137","13,21777","52,42825"
4,49097,11,7,7030303,Grazer Platz,2019,2,15,3,3,...,1,0,0,0,0,0,"796074,3342","5822724,905","13,36007","52,47421"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13385,202775,11,1,1011303,Alexanderplatzviertel,2019,12,15,2,3,...,1,1,0,0,0,0,"799498,019","5828344,449","13,41538","52,52271"
13386,202776,11,1,1011201,Wilhelmstraße,2019,12,14,2,3,...,1,0,0,0,0,0,"797455,1835","5827060,276","13,38421","52,51232"
13387,202777,11,1,1011304,Karl-Marx-Allee,2019,12,14,2,3,...,1,0,0,0,1,0,"799730,882","5827547,138","13,41808","52,51544"
13388,202778,11,11,11030721,Rüdigerstraße,2019,12,18,4,2,...,1,0,1,0,0,0,"805189,971","5827912,648","13,49862","52,51569"


In [4]:
df.shape

(13390, 24)

In [5]:
df.dtypes

OBJECTID        int64
LAND            int64
BEZ             int64
LOR             int64
STRASSE        object
UJAHR           int64
UMONAT          int64
USTUNDE         int64
UWOCHENTAG      int64
UKATEGORIE      int64
UART            int64
UTYP1           int64
ULICHTVERH      int64
IstRad          int64
IstPKW          int64
IstFuss         int64
IstKrad         int64
IstGkfz         int64
IstSonstige    object
USTRZUSTAND    object
LINREFX        object
LINREFY        object
XGCSWGS84      object
YGCSWGS84      object
dtype: object

# Meaning of variables

From the [metadata description](https://www.statistik-berlin-brandenburg.de/opendata/AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2019_Datensatzbeschreibung.pdf) in German, let's explore the data and describe in English what the variables and their values mean.

Land is 11, so Berlin, same in all this data.

BEZ means Bezirk, or neighborhood, described [here](https://www.statistik-berlin-brandenburg.de/regionales/rbs/lor.asp?Kat=4003). The geopandas polygons is not yet present, but we will get it from somewhere else later to be able to count and plot how many accidents are in each neighborhood. The number of accidents in each neighborhood. 

LOR seems to be some area within the neighborhood.

In [6]:
df.BEZ.value_counts()

1     2215
4     1727
2     1381
7     1241
3     1219
8      934
9      892
12     867
6      851
5      798
11     679
10     586
Name: BEZ, dtype: int64

STRASSE = Street Name

UJAHR = year, here only 2019 for all data

UMONAT = month of the year as number from 1 to 12

USTUNDE = hour of the day as number from 0 to 23

UWOCHENTAG = day of the weak, with Sun = 1, Mon = 2, Tue = 3, ..., Fri = 6, Sat = 7

In [7]:
df.UJAHR.value_counts()

2019    13390
Name: UJAHR, dtype: int64

In [8]:
# most accidents in warm months of the year
df.UMONAT.value_counts()

6     1448
8     1371
9     1270
10    1214
5     1207
7     1175
4     1104
11    1081
12     936
3      907
1      891
2      786
Name: UMONAT, dtype: int64

In [9]:
# most accidents in the afternoon
df.USTUNDE.value_counts()

16    1158
15    1120
17    1110
14     969
18     930
13     854
8      800
9      786
12     780
11     718
10     703
7      682
19     681
20     446
6      329
21     319
22     254
23     187
5      163
0      120
1      100
2       62
3       61
4       58
Name: USTUNDE, dtype: int64

In [10]:
# weekdays have more accidents than weekends
df.UWOCHENTAG.value_counts()

3    2248
4    2234
5    2180
2    2131
6    2115
7    1494
1     988
Name: UWOCHENTAG, dtype: int64

Category of how serious is the accident. 

1 = Accident with death
2 = Accident with serious injuries
3 = Accident with light injuries

Most accidents are light, about 7 times less are with serious injuries and from these 60 times rarer are accidents with deaths. 34 accidens per year in Berlin in 2019.

In [11]:
df.UKATEGORIE.value_counts()

3    11374
2     1982
1       34
Name: UKATEGORIE, dtype: int64

UART has 10 categories of how are the actors moving in the accident.

UTYP1 has 7 categories also of how are the actors moving in the accident, but from another perspective.

ULICHTVERH has 3 categories of how dark is it outside: 0 full light; 1 dim light; 2 darkness

IstRad if there is a bike involved: 0 no, 1 yes

IstPKW if there is a personal car involved: 0 no, 1 yes

IstFuss if there is a pedestrian involved: 0 no, 1 yes

IstKrad if there is a motorbike involved: 0 no, 1 yes

IstGkf if there is a truck involved: 0 no, 1 yes

IstSonstig if there is also an actor not mentioned above: 0 no, 1 yes

USTRZUSTAND how humid is the road: 0 not at all (dry), 1 wet as after rain, 2 wet with winter ice or freezing rain

## Coordinates

LINREFX, LINREFY. The coordinate of the accident is given in x, y coordinate in meters for Berlin ETRS89, Zone 32N. 

XGCSWGS84, YGCSWGS84. The coordinate of the accident in longitude, latitude. 

These are in strings with a comma. Let's convert them in numbers by first replacing the comma in a dot and then the string in a float.

In [12]:
df2 = df.copy()
for column in ["LINREFX", "LINREFY", "XGCSWGS84", "YGCSWGS84"]:
    df2[column] = df2[column].map(lambda x : float(x.replace(",", ".")))
df2.head()

,OBJECTID,LAND,BEZ,LOR,STRASSE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,USTRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,49090,11,12,12301203,Wittenau Süd,2019,1,13,6,3,...,1,0,0,0,0,1,794062.2837,5835083.823,13.34146,52.58609
1,49091,11,3,3040818,Pankow Süd,2019,1,9,5,3,...,1,0,0,0,0,0,799130.4007,5832327.415,13.41356,52.55862
2,49093,11,12,12103115,Breitkopfbecken,2019,3,21,6,3,...,0,0,0,0,0,0,795437.6130,5833549.454,13.36034,52.57159
3,49096,11,6,6040703,Nikolassee,2019,1,7,6,2,...,1,1,0,0,0,1,786714.3754,5817042.137,13.21777,52.42825
4,49097,11,7,7030303,Grazer Platz,2019,2,15,3,3,...,1,0,0,0,0,0,796074.3342,5822724.905,13.36007,52.47421


## Check if there are any missing values

Answer: no missing values.

In [13]:
df2.dtypes

OBJECTID         int64
LAND             int64
BEZ              int64
LOR              int64
STRASSE         object
UJAHR            int64
UMONAT           int64
USTUNDE          int64
UWOCHENTAG       int64
UKATEGORIE       int64
UART             int64
UTYP1            int64
ULICHTVERH       int64
IstRad           int64
IstPKW           int64
IstFuss          int64
IstKrad          int64
IstGkfz          int64
IstSonstige     object
USTRZUSTAND     object
LINREFX        float64
LINREFY        float64
XGCSWGS84      float64
YGCSWGS84      float64
dtype: object

In [14]:
# Check if there are columns with missing values We see that none are missing.
for column in df2.columns:
    print(f"column={column} has {df2[column].isnull().sum()} entries with NaN values")

column=OBJECTID has 0 entries with NaN values
column=LAND has 0 entries with NaN values
column=BEZ has 0 entries with NaN values
column=LOR has 0 entries with NaN values
column=STRASSE has 0 entries with NaN values
column=UJAHR has 0 entries with NaN values
column=UMONAT has 0 entries with NaN values
column=USTUNDE has 0 entries with NaN values
column=UWOCHENTAG has 0 entries with NaN values
column=UKATEGORIE has 0 entries with NaN values
column=UART has 0 entries with NaN values
column=UTYP1 has 0 entries with NaN values
column=ULICHTVERH has 0 entries with NaN values
column=IstRad has 0 entries with NaN values
column=IstPKW has 0 entries with NaN values
column=IstFuss has 0 entries with NaN values
column=IstKrad has 0 entries with NaN values
column=IstGkfz has 0 entries with NaN values
column=IstSonstige has 0 entries with NaN values
column=USTRZUSTAND has 0 entries with NaN values
column=LINREFX has 0 entries with NaN values
column=LINREFY has 0 entries with NaN values
column=XGCSWG

## Check for uncleaned data

We notice that these two columns have the significance of integer numbers, but they appear as objects, meaning strings. We need to convert them to numbers. But it does not work directly as some strings look like `812083,7893`. That seems like some corrupted data. So let's investigate. 

### IstSonstige

In [15]:
df2.IstSonstige.value_counts()

0              11508
1               1881
812083,7893        1
Name: IstSonstige, dtype: int64

In [16]:
# check all cases with a comma in the string
def print_value_1(row):
    if "," in row["IstSonstige"]:
        print(row)
    return None
s1 = df2.apply(print_value_1, axis = 1)  

OBJECTID                        201196
LAND                                11
BEZ                                 10
LOR                           10020415
STRASSE        Hellersdorfer Promenade
UJAHR                             2019
UMONAT                              11
USTUNDE                              8
UWOCHENTAG                           4
UKATEGORIE                           3
UART                                 5
UTYP1                                2
ULICHTVERH                           0
IstRad                               0
IstPKW                               1
IstFuss                              0
IstKrad                              0
IstGkfz                              0
IstSonstige                812083,7893
USTRZUSTAND    Hellersdorfer Promenade
LINREFX                    5.83182e+06
LINREFY                        13.6035
XGCSWGS84                      52.5468
YGCSWGS84                            1
Name: 11893, dtype: object


In [17]:
# in this one case, set no strange object was involved (so value zero) 
# and for the rest normal ones, convert string to integer
def transform_value_1(val):
    if "," in val:
        print(val)
        return 0
    else:
        return int(val)
for column in ["IstSonstige"]:
    df2[column] = df2[column].map(lambda x : transform_value_1(x))
df2.IstSonstige.value_counts()

812083,7893


0    11509
1     1881
Name: IstSonstige, dtype: int64

### USTRZUSTAND

In [18]:
df2.USTRZUSTAND.value_counts()

0                          10154
1                           3190
2                             45
Hellersdorfer Promenade        1
Name: USTRZUSTAND, dtype: int64

In [19]:
# check all cases with a comma in the string
def print_value_2(row):
    if " " in row["USTRZUSTAND"]:
        print(row)
    return None
s1 = df2.apply(print_value_2, axis = 1) 

OBJECTID                        201196
LAND                                11
BEZ                                 10
LOR                           10020415
STRASSE        Hellersdorfer Promenade
UJAHR                             2019
UMONAT                              11
USTUNDE                              8
UWOCHENTAG                           4
UKATEGORIE                           3
UART                                 5
UTYP1                                2
ULICHTVERH                           0
IstRad                               0
IstPKW                               1
IstFuss                              0
IstKrad                              0
IstGkfz                              0
IstSonstige                          0
USTRZUSTAND    Hellersdorfer Promenade
LINREFX                    5.83182e+06
LINREFY                        13.6035
XGCSWGS84                      52.5468
YGCSWGS84                            1
Name: 11893, dtype: object


In [20]:
# in this one case, set it is dry, so zero
# and for the rest normal ones, convert string to integer
def transform_value_2(val):
    if " " in val:
        print(val)
        return 0
    else:
        return int(val)
for column in ["USTRZUSTAND"]:
    df2[column] = df2[column].map(lambda x : transform_value_2(x))
df2.IstSonstige.value_counts()

Hellersdorfer Promenade


0    11509
1     1881
Name: IstSonstige, dtype: int64

# Create a GeoDataFrame from the data frame

4326 is just the EPSG identifier of WGS84 (the Earth elipsoid for longitude and latitude)

In [21]:
gdf = gpd.GeoDataFrame(df2, geometry = gpd.points_from_xy(df2.XGCSWGS84, df2.YGCSWGS84, crs = "EPSG:4326"))
gdf.head()

,OBJECTID,LAND,BEZ,LOR,STRASSE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,...,IstFuss,IstKrad,IstGkfz,IstSonstige,USTRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,geometry
0,49090,11,12,12301203,Wittenau Süd,2019,1,13,6,3,...,0,0,0,0,1,794062.2837,5835083.823,13.34146,52.58609,POINT (13.34146 52.58609)
1,49091,11,3,3040818,Pankow Süd,2019,1,9,5,3,...,0,0,0,0,0,799130.4007,5832327.415,13.41356,52.55862,POINT (13.41356 52.55862)
2,49093,11,12,12103115,Breitkopfbecken,2019,3,21,6,3,...,0,0,0,0,0,795437.6130,5833549.454,13.36034,52.57159,POINT (13.36034 52.57159)
3,49096,11,6,6040703,Nikolassee,2019,1,7,6,2,...,1,0,0,0,1,786714.3754,5817042.137,13.21777,52.42825,POINT (13.21777 52.42825)
4,49097,11,7,7030303,Grazer Platz,2019,2,15,3,3,...,0,0,0,0,0,796074.3342,5822724.905,13.36007,52.47421,POINT (13.36007 52.47421)


# Show on an interactive map the 34 deadly accidents

It's an intearctive map, but let's start with it zoomed on Berlin.

We put the longitude and latitude of Berlin from [here](https://dateandtime.info/citycoordinates.php?id=2950159)

In [22]:
gdf_death = gdf[gdf.UKATEGORIE == 1]
gdf_death.head()

,OBJECTID,LAND,BEZ,LOR,STRASSE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,...,IstFuss,IstKrad,IstGkfz,IstSonstige,USTRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,geometry
60,49229,11,1,1011303,Alexanderplatzviertel,2019,2,10,4,1,...,0,0,1,0,0,799571.0120,5828269.929,13.41638,52.52200,POINT (13.41638 52.52200)
175,49811,11,10,10010309,Springpfuhl,2019,1,10,7,1,...,1,0,0,0,1,807656.0523,5829873.139,13.53666,52.53187,POINT (13.53666 52.53187)
916,50859,11,5,5020629,Börnicker Straße,2019,11,17,3,1,...,1,0,0,1,1,785375.0764,5826217.721,13.20597,52.51122,POINT (13.20597 52.51122)
1743,51741,11,2,2040502,Friedenstraße,2019,6,16,5,1,...,1,0,0,0,0,801117.5862,5827836.882,13.43872,52.51727,POINT (13.43872 52.51727)
1955,191208,11,4,4041134,Bismarckallee,2019,1,17,3,1,...,1,0,0,0,1,790860.2576,5823812.355,13.28447,52.48676,POINT (13.28447 52.48676)


In [23]:
folium_map = folium.Map(location = [52.5243700, 13.4105300], tiles="Stamen Terrain", zoom_start=10, width="80%", height="80%")
folium_map

In [24]:
# let's add a marker for each of the accidents
folium.GeoJson(gdf_death).add_to(folium_map)

In [25]:
# display the map
folium_map